# Introduction


Un modèle séquentiel est un modèle qui prend une entrée une séquence d'items et prédit une séquence. Ce type de modèle permet d'effectuer des tâches comme :

* Traduction automatique qui traduit une séquence d'une langue donnée dans une autre
* Sous-titrage d'images
* Résumé de textes
* ..

La plupart des modèles séquentiels sont basés sur une utilisation des réseaux de neurones récurrents. Ce type réseau a des inconvénients entre autres :
* Le temps de calcul est long 
* Les phénomènes de gradient qui disparait et qui explose malgré l'introduction des LSTM ((Long Short-Term Memory Network)
* Ils sont difficilement parallélisable du fait de leur nature
* ..

Pour résoudre ces problèmes, Google a proposé un modèle d'architecture appelé `Transformer` permettant d'éviter la récurrence en utilisant un mécanisme d'`attention`.

# Le mécanisme d'attention

Initialement le mécanisme d'attention a été conçu pour prendre en compte la position des tokens au moment d'une séquence. Il permet de mettre en valeur les parties plus importantes de la séquence. Au moment générer une nouvelle partie de la séquence de sortie, l'attention permet au décodeur de prendre en compte le contexte à partir des vecteurs d'états issus de l'encodage.

Mathématiquement l'attention est une fonction qui prend en entrée une requête et un ensemble de paires clé-valeur fournit en sortie un score. Le score est une somme pondérée des valeurs. La pondération de chaque valeur dépend de la relation entre la requête et la clé correspondantes. Les scores sont normalisés par la fonction `softmax`.

En pratique, cela revient à un calcul matriciel puisqu'on traite simultanément plusieurs requêtes et que chaque requête est représenté par un vecteur (de même que les clés et leurs valeurs).

<img src="images/attention/attention.png">

[Ce code permet d'illuster ce calcul.](http://nlp.seas.harvard.edu/2018/04/03/attention.html) 

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

def attention(query, key, value, mask=None, dropout=None):
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) /  math.sqrt(d_k)
    if mask is not None:
        mask = mask.unsqueeze(1)
        scores = scores.masked_fill(mask == 0, -1e9)
    
    p_attn = F.softmax(scores, dim = -1)
    
    if dropout is not None:
        p_attn = dropout(p_attn)
    
    return torch.matmul(p_attn, value), p_attn

# Multi-Head Attention

On peut associer plusieurs fonctions d'attention. 

<img src="images/attention/multi-head-attention-graph.png">

Cela permet de représenter les vecteurs requêtes, clés et valeurs dans plusieurs sous-espaces.

Les différents projections sont concaténés pour obtenir le résultat final.

<img src="images/attention/multi-head-attention-formule.png">

[Ce code permet d'illuster ce calcul.](http://nlp.seas.harvard.edu/2018/04/03/attention.html) 

In [ ]:
from torch import nn

class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout = 0.1):
        super().__init__()
        
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        
        self.linear_query = nn.Linear(d_model, d_model)
        self.linear_value = nn.Linear(d_model, d_model)
        self.linear_key = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.output = nn.Linear(d_model, d_model)
        selt.attn = None
    
    def forward(self, query, key, value, mask=None):
        bs = query.size(0)
        key = self.linear_key(key).view(bs, -1, self.h, self.d_k)
        query = self.linear_key(query).view(bs, -1, self.h, self.d_k)
        value = self.linear_view(values).view(bs, -1, self.h, self.d_k)
       
        key = key.transpose(1, 2)
        query = query.transpose(1, 2)
        value = value.transpose(1,2)

        scores, self.attn = attention(query, key, value, mask, self.dropout)
        
        concat = scores.transpose(1, 2).contiguous().view(bs, -1, self.d_model)
   
        return self.output(concat)

# Self-attention (auto-attention)

L'auto-attention est très similaire à l'attention mais elle permet intégrer l'interaction entre différentes parties de la séquence.   
Ainsi elle permet de détecter les relations entre les tokens.

Cet exemple présenté par [Google AI](https://ai.googleblog.com/2017/08/transformer-novel-neural-network.html) permet d'illustrer l'application du mécanisme d'auto-attention à la résolution des coréférences.

<img src="images/attention/self-attention.png">

Le mécanisme d'attention permet de faire la différence entre les deux mots `it`.


**Références:**  
[Attention Is All You Need](https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf)  
[Harvard NLP, The Annotated Transformer](http://nlp.seas.harvard.edu/2018/04/03/attention.html)  
[Transformer: A Novel Neural Network Architecture for Language Understanding](https://ai.googleblog.com/2017/08/transformer-novel-neural-network.html)